In [1]:
include("src//hll_store.jl")

Main.Store

In [2]:
using TidierData
using TidierDB 
using DataFrames
using CSV
# using HTTP
using ..Util 

db = TidierDB.connect(:duckdb);
path = "https://gist.githubusercontent.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv"
copy_to(db, path, "mtcars");

TidierDB.@chain db_table(db, :mtcars) begin
    TidierDB.@filter(!starts_with(model, "M"))
    TidierDB.@group_by(cyl)
    TidierDB.@summarize(mpg = mean(mpg))
    TidierDB.@mutate(mpg_squared = mpg^2, 
               mpg_rounded = round(mpg), 
               mpg_efficiency = case_when(
                                 mpg >= cyl^2 , "efficient",
                                 mpg < 15.2 , "inefficient",
                                 "moderate"))            
    TidierDB.@filter(mpg_efficiency in ("moderate", "efficient"))
    TidierDB.@arrange(desc(mpg_rounded))
    TidierDB.@collect
end

TidierDB.@chain db_table(db, :mtcars) begin
    # @distinct(Games)
    # @count
    @collect
end

Row,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
,String?,Float64?,Int64?,Float64?,Int64?,Float64?,Float64?,Float64?,Int64?,Int64?,Int64?,Int64?
1,Mazda RX4,21.0,6,160.0,110,3.9,2.62,16.46,0,1,4,4
2,Mazda RX4 Wag,21.0,6,160.0,110,3.9,2.875,17.02,0,1,4,4
3,Datsun 710,22.8,4,108.0,93,3.85,2.32,18.61,1,1,4,1
4,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
5,Hornet Sportabout,18.7,8,360.0,175,3.15,3.44,17.02,0,0,3,2
6,Valiant,18.1,6,225.0,105,2.76,3.46,20.22,1,0,3,1
7,Duster 360,14.3,8,360.0,245,3.21,3.57,15.84,0,0,3,4
8,Merc 240D,24.4,4,146.7,62,3.69,3.19,20.0,1,0,4,2
9,Merc 230,22.8,4,140.8,95,3.92,3.15,22.9,1,0,4,2


In [3]:
typeof(db)

DuckDB.Connection

In [4]:
using EasyConfig

edge_props = Config()
edge_props["source"] = "file.item"
edge_props["target"] = "assign.item"
edge_props["source_label"] = "csv_file"
edge_props["target_label"] = "csv_column"
edge = Graph.Edge("file.id", "assign.id", "has_column", edge_props)

df = struct_to_df(edge)

Row,source,target,r_type,props
,String,String,String,String
1,"""file.id""","""assign.id""","""has_column""","{""source"":""file.item"",""target"":""assign.item"",""source_label"":""csv_file"",""target_label"":""csv_column""}"


In [5]:
Graph.add_edge(db, edge)

@chain db_table(db, Symbol("information_schema.tables")) begin
    @collect
end

Row,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?
1,memory,main,mtcars,BASE TABLE,missing,missing,missing,missing,missing,YES,NO,missing,missing
2,memory,main,edges,VIEW,missing,missing,missing,missing,missing,NO,NO,missing,missing


In [6]:
@chain db_table(db, :mtcars) begin
    TidierDB.@filter(!starts_with(model, "M"))
    TidierDB.@group_by(cyl)
    TidierDB.@summarize(mpg = mean(mpg))
    TidierDB.@mutate(mpg_squared = mpg^2, 
               mpg_rounded = round(mpg), 
               mpg_efficiency = case_when(
                                 mpg >= cyl^2 , "efficient",
                                 mpg < 15.2 , "inefficient",
                                 "moderate"))            
    TidierDB.@filter(mpg_efficiency in ("moderate", "efficient"))
    TidierDB.@arrange(desc(mpg_rounded))
    TidierDB.@collect
    TidierData.@pivot_longer(everything(), names_to = "variable", values_to = "value")
end

Row,variable,value
,String,Any
1,cyl,4
2,cyl,6
3,mpg,27.3444
4,mpg,19.7333
5,mpg_squared,747.719
6,mpg_squared,389.404
7,mpg_rounded,27.0
8,mpg_rounded,20.0
9,mpg_efficiency,efficient


In [7]:
@chain db_table(db, :mtcars) begin
    TidierDB.@filter(!starts_with(model, "M"))
    TidierDB.@group_by(cyl)
    TidierDB.@summarize(mpg = mean(mpg))
    TidierDB.@mutate(mpg_squared = mpg^2, 
               mpg_rounded = round(mpg), 
               mpg_efficiency = case_when(
                                 mpg >= cyl^2 , "efficient",
                                 mpg < 15.2 , "inefficient",
                                 "moderate"))            
    TidierDB.@filter(mpg_efficiency in ("moderate", "efficient"))
    TidierDB.@arrange(desc(mpg_rounded))
    TidierDB.@show_query
end

WITH cte_1 AS (
SELECT *
	FROM mtcars
	WHERE NOT (starts_with(model, 'M'))),
cte_2 AS (
SELECT cyl, AVG(mpg) AS mpg
	FROM cte_1
	GROUP BY cyl),
cte_3 AS (
SELECT  cyl, mpg, POWER(mpg, 2) AS mpg_squared, ROUND(mpg) AS mpg_rounded, CASE WHEN mpg >= POWER(cyl, 2) THEN 'efficient' WHEN mpg < 15.2 THEN 'inefficient' ELSE 'moderate' END AS mpg_efficiency
	FROM cte_2 ),
cte_4 AS (
SELECT *
	FROM cte_3
	WHERE mpg_efficiency in ('moderate', 'efficient'))  
SELECT *
	FROM cte_4  
	ORDER BY mpg_rounded DESC


In [8]:
@chain db_table(db, :mtcars) begin
    TidierDB.@group_by(cyl)
    TidierDB.@summarize(across((starts_with("a"), ends_with("s")), (mean, sum)))
    TidierDB.@collect
end

Row,cyl,mean_am,mean_vs,sum_am,sum_vs
,Int64?,Float64?,Float64?,Int128?,Int128?
1,4,0.727273,0.909091,8,10
2,6,0.428571,0.571429,3,4
3,8,0.142857,0.0,2,0


In [9]:
using RDatasets
using TidierDB
using TidierData

movies = dataset("ggplot2", "movies");

@chain movies begin
    TidierData.@mutate(Budget = Budget / 1_000_000)
    TidierData.@filter(Budget >= mean(skipmissing(Budget)))
    TidierData.@select(Title, Budget)
    @slice(1:5)
end

Row,Title,Budget
,String,Float64?
1,'Til There Was You,23.0
2,10 Things I Hate About You,16.0
3,102 Dalmatians,85.0
4,13 Going On 30,37.0
5,"13th Warrior, The",85.0


In [10]:
@chain movies begin
    @slice(1:5)
end

Row,Title,Year,Length,Budget,Rating,Votes,R1,R2,R3,R4,R5,R6,R7,R8,R9,R10,MPAA,Action,Animation,Comedy,Drama,Documentary,Romance,Short
,String,Int32,Int32,Int32?,Float64,Int32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…,Int32,Int32,Int32,Int32,Int32,Int32,Int32
1,$,1971,121,missing,6.4,348,4.5,4.5,4.5,4.5,14.5,24.5,24.5,14.5,4.5,4.5,,0,0,1,1,0,0,0
2,$1000 a Touchdown,1939,71,missing,6.0,20,0.0,14.5,4.5,24.5,14.5,14.5,14.5,4.5,4.5,14.5,,0,0,1,0,0,0,0
3,$21 a Day Once a Month,1941,7,missing,8.2,5,0.0,0.0,0.0,0.0,0.0,24.5,0.0,44.5,24.5,24.5,,0,1,0,0,0,0,1
4,"$40,000",1996,70,missing,8.2,6,14.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.5,45.5,,0,0,1,0,0,0,0
5,"$50,000 Climax Show, The",1975,71,missing,3.4,17,24.5,4.5,0.0,14.5,14.5,4.5,0.0,0.0,0.0,24.5,,0,0,0,0,0,0,0


In [11]:
@glimpse(movies)

Rows: 58788
Columns: 24
.Title         String         $, $1000 a Touchdown, $21 a Day Once a Month, $40,
.Year          Int32          1971, 1939, 1941, 1996, 1975, 2000, 2002, 2002, 19
.Length        Int32          121, 71, 7, 70, 71, 91, 93, 25, 97, 61, 99, 96, 10
.Budget        Union{Missing, Int32}missing, missing, missing, missing, missing,
.Rating        Float64        6.4, 6.0, 8.2, 8.2, 3.4, 4.3, 5.3, 6.7, 6.6, 6.0, 
.Votes         Int32          348, 20, 5, 6, 17, 45, 200, 24, 18, 51, 23, 53, 44
.R1            Float64        4.5, 0.0, 0.0, 14.5, 24.5, 4.5, 4.5, 4.5, 4.5, 4.5
.R2            Float64        4.5, 14.5, 0.0, 0.0, 4.5, 4.5, 0.0, 4.5, 4.5, 0.0,
.R3            Float64        4.5, 4.5, 0.0, 0.0, 0.0, 4.5, 4.5, 4.5, 4.5, 4.5, 
.R4            Float64        4.5, 24.5, 0.0, 0.0, 14.5, 14.5, 4.5, 4.5, 0.0, 4.
.R5            Float64        14.5, 14.5, 0.0, 0.0, 14.5, 14.5, 24.5, 4.5, 0.0, 
.R6            Float64        24.5, 14.5, 24.5, 0.0, 4.5, 14.5, 24.5, 14.5, 0.0,
.R7 

In [12]:
@chain begin
    [1, 2, 3]
    filter(isodd, _)
    @aside @info "There are \$(length(_)) elements after filtering"
    sum
end

[ Info: There are $(length(_)) elements after filtering


4

In [13]:
@chain begin
    [1, 2, 3]
    filtered = filter(isodd, _)
    sum
end
filtered == [1, 3]

true

In [14]:
@chain begin
    [1, 2, 3]
    @. sqrt
end

3-element Vector{Float64}:
 1.0
 1.4142135623730951
 1.7320508075688772

In [15]:
using DataFrames, Chain

df = DataFrame(group = [1, 2, 1, 2, missing], weight = [1, 3, 5, 7, missing])

result = @chain df begin
    dropmissing
    filter(r -> r.weight < 6, _)
    groupby(:group)
    combine(:weight => sum => :total_weight)
end

Row,group,total_weight
,Int64,Int64
1,1,6
2,2,3


In [16]:
using CSV

@chain df begin
    dropmissing
    filter(r -> r.weight < 6, _)
    @aside @chain _ begin
            select(:group)
            CSV.write("filtered_groups.csv", _)
        end
    groupby(:group)
    combine(:weight => sum => :total_weight)
end

Row,group,total_weight
,Int64,Int64
1,1,6
2,2,3
